In [1]:
import pandas as pd
import subprocess
import numpy as np
from sklearn.multioutput import MultiOutputRegressor 
from sklearn.externals import joblib
import numpy as np
import pandas as pd
from scipy.stats import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
import xgboost as xgb

## generate kmer matrices

In [12]:
#did kmer = 3,5; fasta = mintrain, val
k_length = 3
fastafile = '../Ernst2DanQ/mintrain.fa'
outname = '../other_features/kmers/3mer_train.csv'

kmerCmd = '../kmer-counter/kmer-counter --fasta --no-rc --k=%d %s' % (k_length, fastafile)
output = subprocess.check_output(kmerCmd, shell=True)

KeyboardInterrupt: 

In [2]:
#While the cell above worked for k = 3 or 5
#had to do it on command line for k=6

prefix = '../other_features/kmers/'
files = ['6mertrain', '6merval', '6mertest']
k_length = 6

In [15]:
for f in files:
    process = subprocess.Popen(['cat', prefix + f + '.txt'], 
                               stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    stdout, stderr = process.communicate()
    x = len(stdout.splitlines()) #num rows
    y = int(4 ** k_length)
    result = pd.DataFrame(pd.np.zeros((x, y)))

    i = 0
    names = []
    cols = {}
    ind = 0
    for line in stdout.splitlines():
        if i % 10000 == 0:
            print(i * 100 / x)
        (header, counts) = line.decode().split('\t')
        header = header[1:]
        kmers = [k for (k,v) in [d.split(':') for d in counts.split(' ')]]
        vals = [int(v) for (k,v) in [d.split(':') for d in counts.split(' ')]]
        
        nums = []
        for k in kmers:
            if k not in names:
                names.append(k)
                cols[k] = ind
                ind += 1
            nums.append(cols[k])
        
        result.loc[i, nums] = vals
        i += 1
        
    result = result.astype(np.int64).loc[:, (result != 0).any(axis=0)]
    result.columns = names
    result = result.reindex(columns = sorted(result.columns))
    result.to_csv(prefix + f + '.csv', index = False)

0.0
2.18735098671403
4.37470197342806
6.562052960142091
8.74940394685612
10.93675493357015
13.124105920284181
15.311456906998211
17.49880789371224
19.68615888042627
21.8735098671403
24.060860853854333
26.248211840568363
28.435562827282393
30.622913813996423
32.81026480071045
34.99761578742448
37.18496677413851
39.37231776085254
41.55966874756657
43.7470197342806
45.93437072099463
48.121721707708666
50.309072694422696
52.496423681136726
54.683774667850756
56.871125654564786
59.058476641278816
61.245827627992846
63.433178614706875
65.6205296014209
67.80788058813494
69.99523157484896
72.182582561563
74.36993354827702
76.55728453499106
78.74463552170508
80.93198650841912
83.11933749513314
85.30668848184717
87.4940394685612
89.68139045527523
91.86874144198926
94.0560924287033
96.24344341541733
98.43079440213135
0.0
50.42101547925175
0.0
98.71668311944718


In [ ]:
#fixing 3mer files
result = result.astype(np.int64).loc[:, (result != 0).any(axis=0)]

#because the script does not use canonical kmers/list the first lexographically, do it myself
if 'TAG' in result.columns:
    print('convert TAG to CTA')
    result = result.rename(index = str, columns = {"TAG": "CTA"})

result = result.reindex(columns = sorted(result.columns))
result.to_csv(outname, index = False)

In [25]:
#fixing 5mer (and 6mer) files
def fix_5mer(filename):
    df = pd.read_csv(filename)
    kmer_list = list(df.columns)

    while kmer_list:
        seq = kmer_list[0]
        newseq = ''
        for i in seq[::-1]:
            if i == 'A':
                newseq = newseq + 'T'
            elif i == 'T':
                newseq = newseq + 'A'
            elif i == 'C':
                newseq = newseq + 'G'
            else:
                newseq = newseq + 'C'
        
        if seq != newseq: #for even kmers, reverse complement may be the same, ex. AAATTT
            sorted_list = [seq, newseq]
            sorted_list.sort() #take canonical/alphabetically first kmer

            if newseq in kmer_list:
                df[sorted_list[0]] += df[sorted_list[1]] #add duplicates
                df.drop(labels = sorted_list[1], axis = 1, inplace = True)
                kmer_list.remove(newseq) #so adding duplicates does not happen twice
            elif sorted_list[0] != seq: #rename to canonical kmer
                df.rename(index = str, columns = {seq: newseq}, inplace = True)
        
        kmer_list.remove(seq)
        
    df = df.reindex(columns = sorted(df.columns))
    print('done')
    return(df)

In [75]:
#train_df = fix_5mer('../other_features/kmers/5mer_train.csv')
#val_df = fix_5mer('../other_features/kmers/5mer_val.csv')
#train_df.to_csv('../other_features/kmers/5mer_train.csv', index = False)
#val_df.to_csv('../other_features/kmers/5mer_val.csv', index = False)
#test_df = fix_5mer('../other_features/kmers/5mer_test.csv')
#test_df.to_csv('../other_features/kmers/5mer_test.csv', index = False)

In [ ]:
#train_df = fix_5mer('../other_features/kmers/6mertrain.csv')
#val_df = fix_5mer('../other_features/kmers/6merval.csv')
#test_df = fix_5mer('../other_features/kmers/6mertest.csv')
#train_df.to_csv('../other_features/kmers/6mertrain.csv', index = False)
#val_df.to_csv('../other_features/kmers/6merval.csv', index = False)
#test_df.to_csv('../other_features/kmers/6mertest.csv', index = False)

In [74]:
files = ['6mertrain', '6merval', '6mertest']
for f in files:
    df = pd.read_csv(prefix + f + '.csv')
    for k in ['GGTACC', 'TCTAGA', 'ATCGAT', 'CGTACG']:
        if k not in list(df.columns):
            df[k] = 0
    df = df.reindex(columns = sorted(df.columns))
    df.to_csv(prefix + f + '.csv', index = False)

## testing

In [3]:
kmer_test = pd.read_csv('../other_features/kmers/6mertest.csv')
kmer_test = kmer_test.values

allscores = pd.read_csv('../train_val_test/zscores/test_zscores.csv', sep="\t", header=None) 
scores = allscores.values
scores_test = scores.astype(float)

In [5]:
#3mer
kmer_train = pd.read_csv('../other_features/kmers/3mer_train.csv')
kmer_train = kmer_train.values
scores_train = pd.read_csv('../train_val_test/zscores/mintrain_zscores.csv', sep="\t", header=None) 
scores_train = scores_train.values

rf1 = MultiOutputRegressor(XGBRegressor(max_depth = 6, n_estimators = 450, random_state = 123, n_jobs=-1, 
                                        silent=False, colsample_bytree = 0.65, gamma = 9.46, reg_lambda = 0.5, 
                                        learning_rate = 0.0112, min_child_weight = 14, 
                                        subsample = 0.6))

rf1.fit(kmer_train, scores_train)
joblib.dump(rf1, '../other_features/kmers/3mer_revise.pkl')

[12:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 4 pruned nodes, max_depth=6
[12:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 6 pruned nodes, max_depth=6
[12:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[12:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 8 pruned nodes, max_depth=6
[12:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 2 pruned nodes, max_depth=6
[12:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 2 pruned nodes, max_depth=6
[12:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 12 pruned nodes, max_depth=6
[12:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 2 pruned nodes, max_depth=6
[12:04:

[12:04:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 12 pruned nodes, max_depth=6
[12:04:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 14 pruned nodes, max_depth=6
[12:04:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 4 pruned nodes, max_depth=6
[12:04:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[12:04:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 14 pruned nodes, max_depth=6
[12:04:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 14 pruned nodes, max_depth=6
[12:04:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 6 pruned nodes, max_depth=6
[12:04:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 10 pruned nodes, max_depth=6
[12

[12:04:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 16 pruned nodes, max_depth=6
[12:04:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 14 pruned nodes, max_depth=6
[12:04:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 16 pruned nodes, max_depth=6
[12:04:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 2 pruned nodes, max_depth=6
[12:04:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 14 pruned nodes, max_depth=6
[12:04:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 18 pruned nodes, max_depth=6
[12:04:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 18 pruned nodes, max_depth=6
[12:04:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 4 pruned nodes, max_depth=6
[12:0

[12:04:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 22 pruned nodes, max_depth=6
[12:04:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 28 pruned nodes, max_depth=6
[12:04:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 10 pruned nodes, max_depth=6
[12:04:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 12 pruned nodes, max_depth=6
[12:04:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 12 pruned nodes, max_depth=6
[12:04:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 28 pruned nodes, max_depth=6
[12:04:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 12 pruned nodes, max_depth=6
[12:04:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 16 pruned nodes, max_depth=6
[12:04:5

[12:05:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 36 pruned nodes, max_depth=6
[12:05:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 22 pruned nodes, max_depth=6
[12:05:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 8 pruned nodes, max_depth=6
[12:05:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 50 pruned nodes, max_depth=6
[12:05:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 18 pruned nodes, max_depth=6
[12:05:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 40 pruned nodes, max_depth=6
[12:05:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 30 pruned nodes, max_depth=6
[12:05:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 20 pruned nodes, max_depth=6
[12:05:0

...

[12:08:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 16 pruned nodes, max_depth=6
[12:08:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 30 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 18 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 12 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 12 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 18 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 20 pruned nodes, max_depth=6
[12:08:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 24 pruned nodes, max_depth=6
[12:08:

[12:08:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 40 pruned nodes, max_depth=6
[12:08:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 42 pruned nodes, max_depth=6
[12:08:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 24 pruned nodes, max_depth=6
[12:08:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 16 pruned nodes, max_depth=6
[12:08:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 32 pruned nodes, max_depth=6
[12:08:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 26 pruned nodes, max_depth=6
[12:08:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 20 pruned nodes, max_depth=6
[12:08:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 22 pruned nodes, max_depth=6
[12:08:5

[12:09:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 42 pruned nodes, max_depth=6
[12:09:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 28 pruned nodes, max_depth=6
[12:09:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 34 pruned nodes, max_depth=6
[12:09:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 30 pruned nodes, max_depth=6
[12:09:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 18 pruned nodes, max_depth=6
[12:09:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 34 pruned nodes, max_depth=6
[12:09:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 20 pruned nodes, max_depth=6
[12:09:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 36 pruned nodes, max_depth=6
[12:09:0

[12:09:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 40 pruned nodes, max_depth=6
[12:09:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 26 pruned nodes, max_depth=6
[12:09:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 20 pruned nodes, max_depth=6
[12:09:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 38 pruned nodes, max_depth=6
[12:09:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 16 pruned nodes, max_depth=6
[12:09:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 20 pruned nodes, max_depth=6
[12:09:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 26 pruned nodes, max_depth=6
[12:09:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 32 pruned nodes, max_depth=6
[12:09:1

['../other_features/kmers/3mer_revise.pkl']

In [10]:
predicted = rf1.predict(kmer_val)
spearmanr = []
for i in range(4):
    spearmanr.append(stats.spearmanr(scores_test[:, i], predicted[:, i]))
print(spearmanr)

[SpearmanrResult(correlation=0.14347842907332842, pvalue=1.0509968177924905e-91), SpearmanrResult(correlation=0.081559765625617536, pvalue=1.2523991216630615e-30), SpearmanrResult(correlation=0.073948844666009941, pvalue=1.8509587710885012e-25), SpearmanrResult(correlation=0.13381719925896127, pvalue=6.5257350594361116e-80)]


In [4]:
rf1 = joblib.load('../other_features/kmers/6mer_xg.pkl')

predicted = rf1.predict(kmer_test)
spearmanr = []
for i in range(4):
    spearmanr.append(stats.spearmanr(scores_test[:, i], predicted[:, i]))
print(spearmanr)

[SpearmanrResult(correlation=0.20402419171062253, pvalue=1.208252772590895e-95), SpearmanrResult(correlation=0.1522898426035868, pvalue=1.282613769769119e-53), SpearmanrResult(correlation=0.1152050841274767, pvalue=2.8183285714022615e-31), SpearmanrResult(correlation=0.16698113091203687, pvalue=3.0365283607235393e-64)]


In [5]:
rf1.estimators_[0].get_params

<bound method XGBModel.get_params of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0.0437, learning_rate=0.0157,
       max_delta_step=0, max_depth=10, min_child_weight=36, missing=nan,
       n_estimators=800, n_jobs=-1, nthread=None, objective='reg:linear',
       random_state=123, reg_alpha=0, reg_lambda=0.8, scale_pos_weight=1,
       seed=None, silent=False, subsample=1)>